# installation

## konlpy를 위한 jdk 설치

https://www.oracle.com/kr/java/technologies/javase/javase8u211-later-archive-downloads.html

해당 링크에서 oracle 가입 및 로그인 후 Windows x64 installer 다운로드 후 설치한다.

윈도우 > 시스템 환경 변수 편집 > 환경변수 > 시스템 변수 > 새로 만들기

변수 : JAVA_HOME         
값(jdk 설치 경로) : C:\Program Files\Java\jdk-1.8

윈도우 > 시스템 환경 변수 편집 > 환경변수 > 시스템 변수 > Path > 편집 > 새로 만들기             
                          
%JAVA_HOME%\bin;

윈도우 cmd에서 $java -version      
으로 설치 확인

## 라이브러리 설치

In [ ]:
!pip install wordcloud
!pip install kss
!pip install konlpy
!pip install scikit-learn
!pip install selenium

# import

In [ ]:
import re
import os
import base64
import math
import numpy as np
from PIL import Image
from collections import Counter
from wordcloud import WordCloud, ImageColorGenerator
import certifi
import matplotlib.pyplot as plt
from matplotlib import font_manager
font_fname = 'C:/Windows/Fonts/malgun.ttf'
font_family = font_manager.FontProperties(fname=font_fname).get_name()
plt.rcParams['font.family'] = font_family
import warnings
warnings.filterwarnings(action='ignore')
import kss
from konlpy.tag import Okt, Kkma
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize 

# 필요한 파일 준비

## stopwords-ko.txt 파일과 같은 경로에 다운로드

## 파일과 같은 경로에 wordcloud_mask.png 이름의 wordcloud 생성을 위한 mask 이미지 저장

# Text에서 stopwords 제거 및 문장 자르기

In [ ]:
class TextAnalyzer:
    def __init__(self):
        self.okt = Okt()
        self.kkma = Kkma()
        self.stopwords = []
        stopwords_path = './stopwords-ko.txt'
        with open(stopwords_path, 'r', encoding='UTF8') as file:
            for line in file:
                self.stopwords.append(line.strip())

    def text2sentences(self, text):
        sentences = kss.split_sentences(text)
        return sentences
        
    def sentences2nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            for word in self.okt.nouns(str(sentence)):
                if word in self.stopwords or len(word) <= 1:
                    continue    
                if word in nouns:
                    continue
                nouns.append(word)
        return nouns

#  단어 분석

In [ ]:
class WordAnalyzer:
    def __init__(self):
        self.text_analyze = TextAnalyzer()
        self.countvec = CountVectorizer()
        self.okt = Okt()
        self.stoptags = ['Determiner', 'Adverb', 'Conjunction', 'Exclamation', 'Josa']
        self.counttags = ['Noun', 'Verb', 'Adjective']
        self.essential_josa = ['은', '는', '이', '가', '을', '를']
        
    def build_word_graph(self, sentence):
        countvec_mat = normalize(self.countvec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.countvec.vocabulary_
        return np.dot(countvec_mat.T, countvec_mat), {vocab[word]: word for word in vocab}
    
    def get_ranks(self, graph, d=0.85):
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0    
            link_sum = np.sum(A[:, id])
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1
            
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B)
        return {idx: r[0] for idx, r in enumerate(ranks)}

    # 키워드
    def text2keywords(self, text, word_num=15):        
        sentences = self.text_analyze.text2sentences(text)
        nouns = self.text_analyze.sentences2nouns(sentences)
        word_graph, idx2word = self.build_word_graph(nouns)
        word_rank_idx = self.get_ranks(word_graph)
        sorted_word_rank_idx = sorted(word_rank_idx, key=lambda k: word_rank_idx[k], reverse=True)
        
        keywords = []
        index = []
        
        for idx in sorted_word_rank_idx[:word_num]:
            index.append(idx)
        for idx in index:
            keywords.append(idx2word[idx])
        return keywords
    
    # 품사 태깅
    def text2postag(self, text):
        postag = self.okt.pos(text)
        return postag
    
    # 빈도수 높은 단어
    def text2countwords(self, text):
        postag = self.text2postag(text)
        countwords_postag = []
        
        for i in range(len(postag)):
            if postag[i][1] in self.counttags:
                countwords_postag.append(postag[i][0])
        return countwords_postag
    
    # visualization
    def words2wordscount(self, words, counttype):
        if counttype == 'individual':
            whole = []
            for i in range(len(words)):
                for j in range(0, len(words)-i, 1):
                    whole.append(words[i])
            words = whole
        else:
            pass

        # 모든 경우에서 count를 통해 dict 생성
        count = Counter(words)
        wordscount = dict(count.most_common())
        # countwoords 선정시 단어 길이, 빈도수 1인 단어 제거
        if counttype == 'count':
            dict_key = list(wordscount.keys())
            for i in dict_key:
                if wordscount[i] == 1:
                    del(wordscount[i])
        else:
            pass
        return wordscount
    


    # 워드클라우드 시각화
    def visualize_wordcloud(self, text, wordtype):
        #wordcloud_path = os.getcwd()+'\\server\\routers\\text_analysis'
        wordcloud_path = '.'
        img_save_path = '.'
        if wordtype == 'keywords':
            mask = np.array(Image.open(wordcloud_path+'/wordcloud_mask.png'))
            words = self.text2keywords(text)
            words = self.words2wordscount(words, 'individual')
        
        elif wordtype == 'countwords':
            mask = np.array(Image.open(wordcloud_path+'/wordcloud_mask.png'))
            words = self.text2countwords(text)
            words = self.words2wordscount(words, 'count')
        
        image_colors = ImageColorGenerator(mask)
        wordcloud = WordCloud(font_path=font_fname, background_color='white',
                              mask=mask, width=mask.shape[1], height=mask.shape[0], prefer_horizontal=0.99999)
        cloud = wordcloud.generate_from_frequencies(words)
        
        plt.figure(figsize=(8,8))
        plt.imshow(cloud.recolor(color_func=image_colors), interpolation='bilinear')
        plt.axis('off')
        plt.savefig(img_save_path+f'/Jasmine_text_wordcloud_{wordtype}.png')





# 실행

In [ ]:
if __name__ == '__main__':
    stttext = '''
    윤석열 대통령이 지난해 11월 캄보디아 프놈펜의 한 호텔에서 열린 한 ·미 정상회담에서 조 바이든 미국 대통령과 악수하고 있다. 
    윤 대통령은 올해 4월 말 미국을 국빈 방문한다. 한국 대통령으로는 12년 만이다. 
    뉴스1         ‘1,2,6,12.’    윤석열 대통령의 4월 말 미국 국빈 방문(State Visit)을 방문을 앞두고 외교가에서 회자되는 숫자들이다.
    ‘1’은 그 어떤 나라의 정상이든 미국 대통령의 4년 임기 중 국빈 방문은 단 1회만 가능하단 뜻이다.
    그만큼 드문 기회로 윤 대통령은 ‘미국의 손님(국빈)’으로 초대돼 의장대 사열과 21발의 예포, 미 영빈관(블레어하우스) 숙박 같은 최고의 예우를 받는다.
    ‘2’는 윤 대통령이 취임 3년 차를 맞은 바이든 대통령의 2번째 국빈이란 의미다.
    첫 국빈은 에마뉘엘 마크롱 프랑스 대통령이었다. ‘6’은 미국을 국빈 방문한 역대 한국 대통령 숫자다. 미국은 혈맹인 한국에도 국빈의 기회를 선별해 부여했다. 이승만·박정희·노태우·김영삼·김대중(DJ)·이명박(MB) 전 대통령이 미국을 국빈 방문했다. ‘12’는 12년 만에 이뤄진 한국 대통령의 미국 국빈방문이란 뜻에서 나왔다. 1998년 당시 미국을 국빈 방문중인던 김대중 전 대통령이 美 하원 본회의장에서 열린 상·하원 합동회의에서 연설을 하기 위해 연단에 오르자 미 의원들이 박수를 치고 있다. 청와대사진기자단         국빈방문은 그 자체로도 화제를 낳을 만큼 외교적 의미가 크다. 역대 국빈방문에선 다양한 에피소드가 쏟아졌고, 화려한 의전 중에서도 양국 간 치열한 외교전이 펼쳐졌다. 그중에서도 기립 박수를 받았던 역대 한국 대통령의 미 상·하원 의회 합동 연설은 국빈 방문의 백미로 꼽힌다. 정부는 이번 방문에서도 윤 대통령의 미 의회 연설을 추진 중이다.    이승만 전 대통령은 6·25 직후인 1954년 미국을 국빈 방문했다. 미국 정부가 제공한 군용기로 워싱턴 D.C 공항에 도착한 이 전 대통령이 밝힌 첫 소감은 “만약에 우리가 조금만 더 용기가 있었다면, 압록강까지 차지할 수 있었다”는 것이었다. 휴전 뒤 분단된 한반도를 용인한 당시 아이젠하워 미국 대통령에 대한 아쉬움이 담긴 표현이었다. 이 전 대통령은 미 의회에선 영어로 “이제 휴전 종결을 선언할 적당한 시기가 됐다(It is quite in place to declare the end of the armistice)”며 재차 미국의 도움을 요청했다. 이승만 전 대통령이 1954년 미국을 국빈방문해 미 의회에서 연설을 하던 모습. 사진 이승만건국대통령기념사업회           박정희 전 대통령도 1965년 린든 B. 존슨 당시 미국 대통령이 보낸 전용기를 타고 미국을 국빈 방문했다. 최대 현안은 한국의 월남 파병이었다. 미국은 박 전 대통령이 머문 블레어하우스에 쌀밥과 김치까지 준비하며 깍듯이 대접했지만, 협상 때는 주한미군 감축 카드를 꺼내 들며 파병을 요구했다. 양국 대통령은 한국의 월남 파병과 미국의 1억 5000만불 장기 차관 및 주한미군 유지가 담긴 공동성명을 발표했다.    반미주의자가 아니냐는 의심을 샀던 DJ는 국빈 방문 중 미 의회 연설을 통해 오해를 씻어내려 했다. DJ는 연설에서 군사 정권이 자신을 죽이려 할 때 미군 헬리콥터가 와서 살려줬다며 “미국은 내게 생명의 은인”이라고 말했다. 미 의회 의원들은 기립 박수로 화답했다. 당시 빌 클린턴 미국 대통령과의 백악관 만찬에선 초대 손님이던 비디오 아티스트 백남준씨가 하의 실종 퍼포먼스를 해 화제가 됐다. 클린턴 대통령의 성추문 스캔들을 풍자했다는 말이 나왔다. 2011년 10월 미국을 국빈 방문했던 이명박 전 대통령이 백악관에서 열린 국빈만찬에서 오바마 당시 미국 대통령과 건배를 하고 있다. 청와대사진기자단         윤 대통령 이전의 마지막 국빈 방문은 2011년 10월 MB였다. 양국 의회의 벽을 넘지 못하던 한·미 자유무역협정(FTA)이 최대 현안이었다. MB 회고록 『대통령의 시간』엔 MB가 당시 버락 오바마 미국 대통령의 국빈방문 요청을 거절한 에피소드가 담겨있다. MB는 “한·미 FTA가 양국 의회서 처리되지 않은 상황에서 국빈 방문은 그림이 안 좋다. 상·하원을 모두 통과된 뒤 가겠다”는 의사를 전했다. 오바마 대통령과 미 의회는 MB의 국빈방문 기간 FTA 법안을 처리했다. MB는 회고록에 “미국에 적극적으로 요청해 국빈방문이 이뤄진 경우가 많았는데 이젠 미국이 먼저 초청해 올 정도로 한국의 국제적 위상이 변화했다”고 썼다.    김재천 서강대 국제대학원 교수는 “국빈방문의 역사는 한국의 발전과도 궤를 같이한다”고 말했다. 김 교수는 “국빈 방문은 화려한 의전만큼이나 성과에 대한 압박도 있다”며 “윤 대통령은 북핵 위협에 대한 확장억제 강화책뿐 아니라 인플레이션 감축법(IRA)과 미국 반도체 법안에 대한 해결책에도 힘을 쏟아야 한다”고 말했다.    김재천 서강대 국제대학원 교수는 “국빈방문의 역사는 한국의 발전과도 궤를 같이한다”고 말했다. 김 교수는 “국빈 방문은 화려한 의전만큼이나 성과에 대한 압박도 있다”며 “윤 대통령은 북핵 위협에 대한 확장억제 강화책뿐 아니라 인플레이션 감축법(IRA)과 미국 반도체 법안에 대한 해결책에도 힘을 쏟아야 한다”고 말했다.
    박태인 기자 park.taein@joongang.co.kr'''
    img_save_path = '.'
    
    TA = TextAnalyzer()     # 텍스트 분석 클래스
    WA = WordAnalyzer()     # 단어 분석 클래스

    keywords      = WA.text2keywords(stttext)                   
    top3_keywords = list(WA.words2wordscount(keywords, 'individual'))[:3]         # 키워드 상위 3개
    countwords      = WA.text2countwords(stttext)                 
    top3_countwords = list(WA.words2wordscount(countwords, 'countwords'))[:3]     # 빈도수 높은 단어 상위 3개
    WA.visualize_wordcloud(stttext, 'keywords')       # 키워드 워드클라우드
    WA.visualize_wordcloud(stttext, 'countwords')     # 빈도수 높은 단어 워드클라우드
